In [1]:
import random
import torch

from datasets import load_dataset
from IPython.display import Audio as AudioDisplay
from torchmetrics.functional.audio.pesq import perceptual_evaluation_speech_quality as pesq
from torchmetrics.functional.audio.stoi import short_time_objective_intelligibility as stoi
from torchmetrics.functional.audio.dnsmos import deep_noise_suppression_mean_opinion_score as dnsmos
from torchmetrics.functional.audio.nisqa import non_intrusive_speech_quality_assessment as nisqa
from torchmetrics.functional.audio import scale_invariant_signal_noise_ratio as si_snr
from torchmetrics.functional.audio import scale_invariant_signal_distortion_ratio as si_sdr

from codec_latent_denoiser import CodecLatentDenoiser, CodecLatentDenoiserProcessor

# Load Model and Dataset

In [2]:
model_path = "gokulkarthik/codec-latent-denoiser-e2"
data_path = "JacobLinCool/VoiceBank-DEMAND-16k"
sampling_rate = 16000

In [3]:
processor = CodecLatentDenoiserProcessor.from_pretrained(model_path)
model = CodecLatentDenoiser.from_pretrained(model_path).eval()

In [4]:
ds = load_dataset("JacobLinCool/VoiceBank-DEMAND-16k", num_proc=32)
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'clean', 'noisy'],
        num_rows: 11572
    })
    test: Dataset({
        features: ['id', 'clean', 'noisy'],
        num_rows: 824
    })
})

In [5]:
def compute_score(preds: torch.Tensor, target: torch.Tensor, sampling_rate: int = 16000) -> dict:
    result = {}
    
    score = si_snr(preds=preds, target=target).item()
    result["si_snr"] = round(score, 2)
    
    score = si_sdr(preds=preds, target=target).item()
    result["si_sdr"] = round(score, 2)
        
    score = stoi(preds=preds, target=target, fs=sampling_rate).item()
    result["stoi"] = round(score, 2)
    
    if sampling_rate in [16000, 8000]:
        mode = "wb" if sampling_rate == 16000 else "nb"
        score = pesq(preds=preds, target=target, fs=sampling_rate, mode=mode).item()
        result["pesq"] = round(score, 1)
    
    score = dnsmos(preds=preds, fs=sampling_rate, personalized=False)[-1].item()
    result["dnsmos"] = round(score, 1)
    
    score = nisqa(preds=preds, fs=sampling_rate)[0].item()
    result["nisqa"] = round(score, 1)

    return result

# Test sample

In [6]:
sample_idx = random.randint(0, len(ds['test']))
sample = ds['test'][sample_idx]
clean = torch.from_numpy(sample['clean']['array'])
noisy = torch.from_numpy(sample['noisy']['array'])
print(clean.shape, noisy.shape)
sample

torch.Size([25599]) torch.Size([25599])


{'id': 'p232_100',
 'clean': {'path': 'p232_100.wav',
  'array': array([-0.00030518, -0.00042725, -0.00030518, ...,  0.00314331,
          0.00305176,  0.00286865], shape=(25599,)),
  'sampling_rate': 16000},
 'noisy': {'path': 'p232_100.wav',
  'array': array([0.04364014, 0.06744385, 0.04751587, ..., 0.12585449, 0.12817383,
         0.11587524], shape=(25599,)),
  'sampling_rate': 16000}}

In [7]:
AudioDisplay(clean, rate=sampling_rate)

In [8]:
AudioDisplay(noisy, rate=sampling_rate)

In [9]:
print(compute_score(preds=noisy, target=clean))

{'si_snr': 0.27, 'si_sdr': 0.27, 'stoi': 0.88, 'pesq': 2.2, 'dnsmos': 2.3, 'nisqa': 3.3}


In [10]:
print(compute_score(preds=noisy, target=noisy))
print(compute_score(preds=clean, target=clean))
print(compute_score(preds=clean, target=noisy))

{'si_snr': 179.93, 'si_sdr': 179.93, 'stoi': 1.0, 'pesq': 4.6, 'dnsmos': 2.3, 'nisqa': 3.3}
{'si_snr': 177.04, 'si_sdr': 177.04, 'stoi': 1.0, 'pesq': 4.6, 'dnsmos': 3.0, 'nisqa': 4.3}
{'si_snr': 0.27, 'si_sdr': 0.27, 'stoi': 0.79, 'pesq': 1.5, 'dnsmos': 3.0, 'nisqa': 4.3}


# Codec Latent Denoiser

In [11]:
with torch.inference_mode():
    inputs = processor(noisy)["input_values"]
    outputs = model(inputs, denoise=False, decode=True)
    outputs_denoised = model(inputs, denoise=True, decode=True)
    print("noisy:", noisy.shape)
    print("inputs:", inputs.shape)
    print("audio_embeddings:", outputs_denoised.audio_embeddings.shape)
    print("audio_generated:", outputs_denoised.audio_generated.shape)

noisy: torch.Size([25599])
inputs: torch.Size([1, 1, 25600])
audio_embeddings: torch.Size([1, 1024, 80])
audio_generated: torch.Size([1, 1, 25600])


In [12]:
noisy_generated = torch.zeros_like(noisy)
noisy_denoised_generated = torch.zeros_like(noisy)
T_min = min(outputs_denoised.audio_generated.shape[-1], noisy.shape[-1])
noisy_generated[:T_min] = outputs.audio_generated[0][0][:T_min]
noisy_denoised_generated[:T_min] = outputs_denoised.audio_generated[0][0][:T_min]

In [13]:
AudioDisplay(clean, rate=sampling_rate)

In [14]:
AudioDisplay(noisy, rate=sampling_rate)

In [15]:
AudioDisplay(noisy_generated, rate=sampling_rate)

In [16]:
AudioDisplay(noisy_denoised_generated, rate=sampling_rate)

In [17]:
print(compute_score(preds=noisy, target=clean))
print(compute_score(preds=noisy_generated, target=clean))
print(compute_score(preds=noisy_denoised_generated, target=clean))

{'si_snr': 0.27, 'si_sdr': 0.27, 'stoi': 0.88, 'pesq': 2.2, 'dnsmos': 2.3, 'nisqa': 3.3}
{'si_snr': -7.67, 'si_sdr': -7.68, 'stoi': 0.85, 'pesq': 2.1, 'dnsmos': 2.6, 'nisqa': 3.4}
{'si_snr': -6.17, 'si_sdr': -6.17, 'stoi': 0.86, 'pesq': 2.5, 'dnsmos': 2.9, 'nisqa': 4.5}
